In [ ]:
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#8F003C','#eb3446','Tourney','Smokum',45,10
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h4 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h4>"""%string))
    
    
dhtml('🔥💥 TPS June 2022 IterativeImputer Baseline 🔥💥' )

## Importing Libraries

In [ ]:
# Basic Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Metric Import
from sklearn.metrics import mean_squared_error #squared = False for RMSE

# Model Imports
import xgboost
from lightgbm import LGBMRegressor as lgbm
from catboost import CatBoostRegressor as cb
from sklearn.ensemble import RandomForestRegressor

# IterativeImputer Import
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Free Memory using gc.collect()
import gc


In [ ]:
# pip install sweetviz
# import sweetviz as sv

## Loading in the Data and Datatype conversions

In [ ]:
df = pd.read_csv("../input/tabular-playground-series-jun-2022/data.csv")
ss = pd.read_csv("../input/tabular-playground-series-jun-2022/sample_submission.csv")

In [ ]:
# Datatype Conversions
for i in df.columns:
    if df[i].dtypes == 'float64':
        df[i] = df[i].astype('float16')
    elif df[i].dtypes == 'int64':
        df[i] = df[i].astype('int16')

In [ ]:
# my_report = sv.analyze(df)
# my_report.show_html() # Default arguments will generate to "SWEETVIZ_REPORT.html"

In [ ]:
df.head()

In [ ]:
df.shape

## Looking at the Null Data

In [ ]:
df_null = df.copy()
for i in df.columns:
    if (df[i].isnull().any() == False):
        df_null = df_null.drop(i,axis = 'columns')

In [ ]:
df_null.shape

In [ ]:
# Almost same number of null values (2%) in every feature with null values
print(df_null.isnull().sum().max())
print(df_null.isnull().sum().min())

## Some Data Analysis

I was about to write the full analysis when I noticed that some of the basic things that this baseline approach is based on have been covered in this discussion: 

https://www.kaggle.com/competitions/tabular-playground-series-jun-2022/discussion/328420

## First Iterative Imputation of F_4 features (Highly Correlated)

In [ ]:
cols_4 = []
for i in df.columns:
    if '4' in i.split('_')[1]:
        cols_4.append(i)
cols_4.append('row_id')

In [ ]:
imputer = IterativeImputer(estimator= xgboost.XGBRegressor(n_estimators=1000),max_iter=20, verbose = 2)
imputed_df_4 = pd.DataFrame(imputer.fit_transform(df[cols_4]), columns = cols_4) 

In [ ]:
df = df.drop(cols_4, axis = 'columns')
df = pd.concat([df,imputed_df_4],axis = 'columns')

## Freeing up Memory using gc.collect()

In [ ]:
del imputed_df_4
gc.collect()

In [ ]:
del df_null
gc.collect()

## Final Iterative Imputation

In [ ]:
imputer = IterativeImputer(estimator= xgboost.XGBRegressor(n_estimators=1000),max_iter=20,verbose = 2)
imputed_df = pd.DataFrame(imputer.fit_transform(df), columns = df.columns) 

## Submission

In [ ]:
for i, row in enumerate(ss.values):
    row_col = row[0]
    imputed_row = row_col.split("-")[0] #get the row index
    imputed_col = row_col.split("-")[1] #get the column index
    ss.at[i, "value"] = imputed_df.iloc[int(imputed_row)][imputed_col]

In [ ]:
ss.head()

In [ ]:
ss.to_csv("submission.csv", index=False)

In [ ]:
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#000CA4','#eb3446','Big Shoulders Inline Text','Smokum',50,10
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h4 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h4>"""%string))
    
    
dhtml(' Do UPVOTE if you like my work ' )